# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6658a0b970>
├── 'a' --> tensor([[ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784]])
└── 'x' --> <FastTreeValue 0x7f6658a0b9d0>
    └── 'c' --> tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                        [ 0.2373, -0.7642, -0.4383, -0.3825],
                        [ 0.5687,  1.0945,  0.2058,  0.3947]])

In [4]:
t.a

tensor([[ 0.4008,  0.9582, -0.0116],
        [-0.9696, -1.0183,  1.3784]])

In [5]:
%timeit t.a

69.3 ns ± 0.121 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6658a0b970>
├── 'a' --> tensor([[-0.9477, -0.0034, -1.9295],
│                   [-0.3496,  0.4022, -0.1957]])
└── 'x' --> <FastTreeValue 0x7f6658a0b9d0>
    └── 'c' --> tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                        [ 0.2373, -0.7642, -0.4383, -0.3825],
                        [ 0.5687,  1.0945,  0.2058,  0.3947]])

In [7]:
%timeit t.a = new_value

66.3 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4008,  0.9582, -0.0116],
               [-0.9696, -1.0183,  1.3784]]),
    x: Batch(
           c: tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4008,  0.9582, -0.0116],
        [-0.9696, -1.0183,  1.3784]])

In [11]:
%timeit b.a

60.4 ns ± 0.0246 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7528,  0.4932, -0.2619],
               [-1.1644,  0.0498,  1.1516]]),
    x: Batch(
           c: tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.136 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

857 ns ± 0.255 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 31.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f65ae665f70>
├── 'a' --> tensor([[[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]],
│           
│                   [[ 0.4008,  0.9582, -0.0116],
│                    [-0.9696, -1.0183,  1.3784]]])
└── 'x' --> <FastTreeValue 0x7f65ae665e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f65ae665f40>
├── 'a' --> tensor([[ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784],
│                   [ 0.4008,  0.9582, -0.0116],
│                   [-0.9696, -1.0183,  1.3784]])
└── 'x' --> <FastTreeValue 0x7f65ae64a430>
    └── 'c' --> tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                        [ 0.2373, -0.7642, -0.4383, -0.3825],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 43.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2059,  0.1709,  0.4490, -0.9442],
                       [ 0.2373, -0.7642, -0.4383, -0.3825],
                       [ 0.5687,  1.0945,  0.2058,  0.3947]],
              
                      [[ 1.2059,  0.1709,  0.4490, -0.9442],
                       [ 0.2373, -0.7642, -0.4383, -0.3825],
                       [ 0.5687,  1.0945,  0.2058,  0.3947]],
              
                      [[ 1.2059,  0.1709,  0.4490, -0.9442],
                       [ 0.2373, -0.7642, -0.4383, -0.3825],
                       [ 0.5687,  1.0945,  0.2058,  0.3947]],
              
                      [[ 1.2059,  0.1709,  0.4490, -0.9442],
                       [ 0.2373, -0.7642, -0.4383, -0.3825],
                       [ 0.5687,  1.0945,  0.2058,  0.3947]],
              
                      [[ 1.2059,  0.1709,  0.4490, -0.9442],
                       [ 0.2373, -0.7642, -0.4383, -0.3825],
                       [ 0.5687,  1.0945,  0.2058,  0.3947]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947],
                      [ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947],
                      [ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947],
                      [ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947],
                      [ 1.2059,  0.1709,  0.4490, -0.9442],
                      [ 0.2373, -0.7642, -0.4383, -0.3825],
                      [ 0.5687,  1.0945,  0.2058,  0.3947],
                      [ 1.2059,  0.1709,  0.4490, -0.9442],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 4.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
